# Imports

In [1]:
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import (TimeoutException)
import urllib
import time
import pandas as pd
import openpyxl
import os
from dotenv import load_dotenv 

# Paths e Login

In [2]:
user_path = '//*[@id="username"]'
senha_path = '//*[@id="password"]'
botao_login_path = '//*[@id="loginbtn"]'

iframe_xpath = "//iframe[contains(@src, 'youtube.com') or contains(@src, 'youtu.be')]"

user  = os.getenv("VIRTUS_USER")
senha = os.getenv("VIRTUS_PASSWORD")

# Main Code

In [3]:


class VIRTUS:
    def __init__(self) -> None:
        """
        Este método construtor para fazer login na plataforma
        e conseguir ter acesso a aba de aulas.
        """
        self.base_url = "https://moodle.virtus-cc.ufcg.edu.br/"
        options = ChromeOptions()
        options.add_argument("--start-maximized")
        
        try:
            service = Service(ChromeDriverManager().install())
        except ValueError:
            latest_chromedriver_version_url = "https://chromedriver.storage.googleapis.com/LATEST_RELEASE"
            latest_chromedriver_version = urllib.request.urlopen(latest_chromedriver_version_url).read().decode('utf-8')
            service = Service(ChromeDriverManager(version=latest_chromedriver_version).install())

        self.browser = Chrome(service=service, options=options)
        self.browser.get(self.base_url)
        
        WebDriverWait(self.browser, 5).until(
            EC.element_to_be_clickable(
                (By.XPATH, user_path)
            )
        ).send_keys(user)

        WebDriverWait(self.browser, 5).until(
            EC.element_to_be_clickable(
                (By.XPATH, senha_path)
            )
        ).send_keys(senha)

        actions = ActionChains(self.browser)
        actions.send_keys(Keys.ENTER)
        actions.perform()

        time.sleep(2)
        self.extract_data()
        self.extract_sinc()


    def extract_data(self):
        """
        Este método usa um dicionário de configuração para percorrer módulos e aulas,
        extrai os links, e ao final, cria e salva uma tabela Excel.
        """

        # Dicionário de Configuração 
        config_aulas_por_modulo = {
            1: 4,
            2: 4,
            3: 4,
            4: 3,  
            5: 4
        }
        
        dados_coletados = []
        guia_principal_moodle = self.browser.current_window_handle
        
        
        # O método .items() nos dá acesso tanto à chave (numero_modulo) quanto ao valor (total_aulas)
        for numero_modulo, total_aulas in config_aulas_por_modulo.items():
            print(f"\n{'='*15} INICIANDO MÓDULO {numero_modulo} (Total de Aulas: {total_aulas}) {'='*15}")
            
            # O laço de interno usa o valor vindo do dicionário
            for numero_aula in range(1, total_aulas + 1):
                print(f"\n--- Procurando Aula {numero_aula} do Módulo {numero_modulo} ---")

                self.browser.switch_to.window(guia_principal_moodle)
                section_id = numero_modulo + 1
                link_modulo = f'https://moodle.virtus-cc.ufcg.edu.br/course/view.php?id=199&section={section_id}'
                self.browser.get(link_modulo)
                
                try:
                    aula_texto = f'[Vídeo] Aula {numero_aula}'
                    aula_link_xpath = f"//li[contains(@class, 'activity') and .//span[contains(text(), '{aula_texto}')]]//a"
                    
                    print(f"Procurando o link para: '{aula_texto}'")
                    elemento_link_aula = WebDriverWait(self.browser, 10).until(
                        EC.presence_of_element_located((By.XPATH, aula_link_xpath))
                    )
                    
                    link_pagina_video = elemento_link_aula.get_attribute('href')
                    self.browser.get(link_pagina_video)
                    
                except TimeoutException:
                    print(f"AVISO: Não foi possível encontrar o link para a '{aula_texto}'. Pulando para a próxima.")
                    continue

                try:
                    print("Na página da aula, procurando o iframe do vídeo...")
                    iframe_xpath = "//iframe[contains(@src, 'youtube.com') or contains(@src, 'youtu.be')]"
                    iframe_do_video = WebDriverWait(self.browser, 10).until(
                        EC.presence_of_element_located((By.XPATH, iframe_xpath))
                    )
                    
                    link_embed = iframe_do_video.get_attribute('src')
                    id_com_parametros = link_embed.split('/')[-1]
                    video_id = id_com_parametros.split('?')[0]
                    link_final_yt = f"https://www.youtube.com/watch?v={video_id}"
                    print(f"Link do YouTube convertido: {link_final_yt}")
                    
                    linha_de_dados = {
                        'Módulo': numero_modulo,
                        'Nº da Aula': numero_aula,
                        'Link': link_final_yt
                    }
                    dados_coletados.append(linha_de_dados)
                    print("Dados da aula adicionados à lista.")

                    self.browser.switch_to.new_window('tab')
                    self.browser.get(link_final_yt)
                    print(f"Página da '{aula_texto}' no YouTube carregada.")
                    time.sleep(5) 
                    self.browser.close()
                    self.browser.switch_to.window(guia_principal_moodle)
                    print("Guia do YouTube fechada. Foco retornado para o Moodle.")

                except TimeoutException:
                    print(f"ERRO: Não encontrou o iframe do vídeo para a '{aula_texto}'.")
                except Exception as e:
                    print(f"ERRO inesperado na extração do vídeo: {e}")

        print(f"\n{'='*15} PROCESSO DE COLETA FINALIZADO {'='*15}")

        if dados_coletados:
            print("\nCriando tabela com os dados coletados...")
            df = pd.DataFrame(dados_coletados)
            nome_arquivo = "links.xlsx"
            df.to_excel(nome_arquivo, index=False)
            print(f"Sucesso! A tabela foi salva no arquivo '{nome_arquivo}'")
        else:
            print("\nNenhum dado foi coletado para salvar.")

    def extract_sinc(self):
        """
        Este método percorre os encontros síncronos, extrai os links,
        adiciona um link de apresentação e salva tudo em uma tabela Excel.
        """
     
        apresentacao_link = 'https://www.youtube.com/watch?v=cIt7J0ijOlU&ab_channel=VIRTUS'

        aulas_sincronas = {
            1: 1, 2: 1, 3: 1, 4: 1, 5: 1
        }
        
        dados_coletados = []
        guia_principal_moodle = self.browser.current_window_handle
        
        # O método .items() nos dá acesso tanto à chave (numero_modulo) quanto ao valor (total_aulas)
        for numero_modulo, total_aulas in aulas_sincronas.items():
            
            # O laço interno usa o valor vindo do dicionário
            for numero_aula in range(1, total_aulas + 1):
                print(f"\n--- Procurando Encontro Síncrono do Módulo {numero_modulo} ---")

                self.browser.switch_to.window(guia_principal_moodle)
                section_id = numero_modulo + 1
                link_modulo = f'https://moodle.virtus-cc.ufcg.edu.br/course/view.php?id=199&section={section_id}'
                self.browser.get(link_modulo)
                
                try:
                    aula_texto = 'Encontro Síncrono'
                    aula_link_xpath = f"//li[contains(@class, 'activity') and .//span[contains(text(), '{aula_texto}')]]//a"
                    
                    print(f"Procurando o link para: '{aula_texto}'")
                    elemento_link_aula = WebDriverWait(self.browser, 10).until(
                        EC.presence_of_element_located((By.XPATH, aula_link_xpath))
                    )
                    
                    link_pagina_video = elemento_link_aula.get_attribute('href')
                    self.browser.get(link_pagina_video)
                    
                except TimeoutException:
                    print(f"AVISO: Não foi possível encontrar o link para a '{aula_texto}' no Módulo {numero_modulo}.")
                    continue

                try:
                    print("Na página da aula, procurando o iframe do vídeo...")
                    iframe_xpath = "//iframe[contains(@src, 'youtube.com') or contains(@src, 'youtu.be')]"
                    iframe_do_video = WebDriverWait(self.browser, 10).until(
                        EC.presence_of_element_located((By.XPATH, iframe_xpath))
                    )
                    
                    link_embed = iframe_do_video.get_attribute('src')
                    id_com_parametros = link_embed.split('/')[-1]
                    video_id = id_com_parametros.split('?')[0]
                    link_final_yt = f"https://www.youtube.com/watch?v={video_id}"
                    print(f"Link do YouTube convertido: {link_final_yt}")
                    
                    linha_de_dados = {
                        'Módulo': numero_modulo,
                        'Nº da Aula': numero_aula,
                        'Link': link_final_yt
                    }
                    dados_coletados.append(linha_de_dados)
                    print("Dados do encontro adicionados à lista.")

                    # Lógica para abrir e fechar a guia
                    self.browser.switch_to.new_window('tab')
                    self.browser.get(link_final_yt)
                    print(f"Página da '{aula_texto}' no YouTube carregada.")
                    time.sleep(5) 
                    self.browser.close()
                    self.browser.switch_to.window(guia_principal_moodle)
                    print("Guia do YouTube fechada. Foco retornado para o Moodle.")

                except TimeoutException:
                    print(f"ERRO: Não encontrou o iframe do vídeo para a '{aula_texto}'.")
                except Exception as e:
                    print(f"ERRO inesperado na extração do vídeo: {e}")

        print(f"\n{'='*15} PROCESSO DE COLETA FINALIZADO {'='*15}")

        print("\nAdicionando o link da apresentação geral...")
        linha_apresentacao = {
            'Módulo': 'Apresentação',
            'Nº da Aula': 0,
            'Link': apresentacao_link
        }
        dados_coletados.append(linha_apresentacao)


        if dados_coletados:
            print("\nCriando tabela com todos os dados coletados...")
            df = pd.DataFrame(dados_coletados)
            nome_arquivo = "links_sincronos.xlsx"
            
            df.to_excel(nome_arquivo, index=False)
            print(f"Sucesso! A tabela foi salva no arquivo '{nome_arquivo}'")
        else:
            print("\nNenhum dado foi coletado para salvar.")
        
        self.browser.close()

virtus = VIRTUS()


=============== INICIANDO MÓDULO 1 (Total de Aulas: 4) ===============

--- Procurando Aula 1 do Módulo 1 ---
Procurando o link para: '[Vídeo] Aula 1'
Na página da aula, procurando o iframe do vídeo...
Link do YouTube convertido: https://www.youtube.com/watch?v=HBDbPns1Qyw
Dados da aula adicionados à lista.
Página da '[Vídeo] Aula 1' no YouTube carregada.
Guia do YouTube fechada. Foco retornado para o Moodle.

--- Procurando Aula 2 do Módulo 1 ---
Procurando o link para: '[Vídeo] Aula 2'
Na página da aula, procurando o iframe do vídeo...
Link do YouTube convertido: https://www.youtube.com/watch?v=GpXSae0ydZA
Dados da aula adicionados à lista.
Página da '[Vídeo] Aula 2' no YouTube carregada.
Guia do YouTube fechada. Foco retornado para o Moodle.

--- Procurando Aula 3 do Módulo 1 ---
Procurando o link para: '[Vídeo] Aula 3'
Na página da aula, procurando o iframe do vídeo...
Link do YouTube convertido: https://www.youtube.com/watch?v=e1r7m-r-92Y
Dados da aula adicionados à lista.
Página

# Baixando Vídeos 

In [4]:
import pandas as pd
from pytubefix import YouTube
from pytubefix.cli import on_progress
import os

def download_video(url, resolucao_desejada, pasta="."):
    """
    Baixa um vídeo do YouTube em uma resolução específica.

    Args:
        url (str): O link do vídeo do YouTube.
        resolucao_desejada (str): A resolução desejada (ex: '1080p', '720p').
        pasta (str): A pasta onde o vídeo será salvo.
    """
    try:
        print("\n" + "="*50)
        print(f"Tentando baixar o vídeo: {url}")

        yt = YouTube(url, on_progress_callback=on_progress)
        print(f"Título: {yt.title}")
        print(f"Resolução desejada: {resolucao_desejada}")

        stream = yt.streams.filter(res=resolucao_desejada, progressive=True, file_extension='mp4').first()

        if not stream:
            print(f"AVISO: Resolução {resolucao_desejada} não encontrada com áudio e vídeo juntos.")
            print("Buscando a maior resolução progressiva disponível...")
            stream = yt.streams.get_highest_resolution()
        
        if not stream:
            print("ERRO: Nenhum stream de vídeo disponível para download.")
            return 

        print(f"Stream selecionado: {stream.resolution} ({round(stream.filesize / 1024 / 1024, 2)} MB)")
        
        print("Iniciando o download...")
        stream.download(output_path=pasta)
        print(f"Sucesso! Download de '{yt.title}' concluído.")

    except Exception as e:
        print(f"ERRO ao baixar o vídeo {url}. Motivo: {e}")

def main():
    """
    Lê as tabelas do Excel e coordena o download dos vídeos.
    """

    arquivo_aulas = "links.xlsx"
    arquivo_sincronos = "links_sincronos.xlsx"

    # Processando as aulas normais (1080p)
    if os.path.exists(arquivo_aulas):
        print(f"\n\n{'#'*20} INICIANDO DOWNLOADS DAS AULAS (1080p) {'#'*20}")
        df_aulas = pd.read_excel(arquivo_aulas)
    
        for link in df_aulas['Link']:
            download_video(link, resolucao_desejada='1080p', pasta="Aulas_Assincronas")
    else:
        print(f"AVISO: Arquivo '{arquivo_aulas}' não encontrado. Pulando downloads.")

    # Processando as aulas síncronas (720p)
    if os.path.exists(arquivo_sincronos):
        print(f"\n\n{'#'*20} INICIANDO DOWNLOADS DOS ENCONTROS SÍNCRONOS (720p) {'#'*20}")
        df_sincronos = pd.read_excel(arquivo_sincronos)
        
        for link in df_sincronos['Link']:
            download_video(link, resolucao_desejada='720p', pasta="Aulas_Sincronas")
    else:
        print(f"AVISO: Arquivo '{arquivo_sincronos}' não encontrado. Pulando downloads.")

    print("\n\nProcesso de download finalizado!")

if __name__ == "__main__":
    main()



#################### INICIANDO DOWNLOADS DAS AULAS (1080p) ####################

Tentando baixar o vídeo: https://www.youtube.com/watch?v=HBDbPns1Qyw
Título: VIRTUS-CC | FCRH | FPSII | Módulo 01 - Aula 01
Resolução desejada: 1080p
AVISO: Resolução 1080p não encontrada com áudio e vídeo juntos.
Buscando a maior resolução progressiva disponível...
Stream selecionado: 360p (31.99 MB)
Iniciando o download...
Sucesso! Download de 'VIRTUS-CC | FCRH | FPSII | Módulo 01 - Aula 01' concluído.

Tentando baixar o vídeo: https://www.youtube.com/watch?v=GpXSae0ydZA
Título: VIRTUS-CC | FCRH | FPSII | Módulo 01 - Aula 02
Resolução desejada: 1080p
AVISO: Resolução 1080p não encontrada com áudio e vídeo juntos.
Buscando a maior resolução progressiva disponível...
Stream selecionado: 360p (22.26 MB)
Iniciando o download...
Sucesso! Download de 'VIRTUS-CC | FCRH | FPSII | Módulo 01 - Aula 02' concluído.

Tentando baixar o vídeo: https://www.youtube.com/watch?v=e1r7m-r-92Y
Título: VIRTUS-CC | FCRH | FPS